In [3]:
from torchnet.meter import mAPMeter
import torch

import numpy as np

In [75]:
class MetricMeter(object):
    def __init__(self, class_num=20):
        self.mAPMeter = mAPMeter()
        self.true_array = np.zeros((1, class_num))
        self.total = 0
        self.cnt_array = np.zeros((1, class_num)) # count occurrence of labels
        
    def add(self, out, target):
        '''
        output: B*K
        target: B*K
        '''
        self.mAPMeter.add(out, target)
        if isinstance(out, torch.cuda.FloatTensor):
            output = out.cpu().numpy() > 0.5
        elif isinstance(out, torch.FloatTensor):
            output = out.numpy() > 0.5
        else:
            output = out > 0.5
        result = (target.astype(bool) == output).astype(int) # if true result=1 else res=0
        self.true_array += np.sum(result, axis=0)
        self.cnt_array += np.sum(target, axis=0)
        self.total += out.shape[0]
        
    def value(self):
        mAcc = np.mean(self.true_array.astype(float) / self.total)
        wAcc = np.sum(self.true_array.astype(float) / self.total * self.cnt_array / np.sum(self.cnt_array))
        mAP = self.mAPMeter.value()
        return {'mAcc': mAcc, 'wAcc': wAcc, 'mAP': mAP}
    
    def reset(self):
        self.mAPMeter.reset()
        self.true_array = np.zeros(1, class_num)
        self.total = 0
        self.cnt_array = np.zeros(1, class_num) # count occurrence of labels

In [76]:
m = MetricMeter(1)

In [97]:
tmp_out = np.array([0.2, 0.3, 0.4, 0.5, 0.9])
tmp_out = np.reshape(tmp_out, (5, 1))

tmp_target = np.array([0, 0, 0, 0, 0], dtype=int)
tmp_target = np.reshape(tmp_target, (5, 1))
m.add(tmp_out, tmp_target)
m.value()

In [120]:
m.add(tmp_out, tmp_target)
m.value()

{'mAcc': 0.806060606060606, 'wAcc': 0.806060606060606, 'mAP': tensor(0.0694)}

In [65]:
tmp_out = np.random.uniform(size=(1, 20))
tmp_target = np.random.randint(low=0, high=2, size=(1, 20), dtype=int)
m.add(tmp_out, tmp_target)

In [68]:
np.reshape?

In [55]:
(tmp_out > 0.5) == tmp_target

array([[ True,  True, False,  True, False,  True, False,  True,  True,
         True, False, False,  True, False, False, False,  True,  True,
        False, False]])